In [3]:
import os
import pickle
import numpy as np
import pandas as pd
import nibabel as nib
import tensorflow as tf

from PIL import Image
from scipy import ndimage
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split

In [2]:
home = os.path.expanduser('~')
base = os.path.join(home, 'Datasets', 'ImageCLEF')

dataset_dir = os.path.join(base, 'Mean_Slice_Masks')

In [5]:
img = Image.open(os.path.join(dataset_dir, 'test', '1', 'TRN_0255_Mean_Slice_Mask.png'))

In [6]:
print(img.size)

(292, 462)


In [2]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu_devices[0], True)

# tf.config.gpu.set_per_process_memory_growth(True)

In [3]:
model_name = '3d_image_classification_normalized'
# 3d_image_classification_normalized_history.
random_state=1

## Directory structure:
* **Dataset Directory**: $HOME/Datasets/ImageCLEF/
* extracted .nii.gz files are in a Dataset subfolder in Dataset Directory
* metadata file is in the Dataset Directory

In [4]:
home = os.path.expanduser('~')
base = os.path.join(home, 'Datasets', 'ImageCLEF')

dataset_dir = os.path.join(base, 'Dataset')

label_path = os.path.join(base, '4231cdb3-af46-4674-be08-95b904a62093_TrainSet_metaData.csv')
df = pd.read_csv(label_path)

df.head(10)

,FileName,TypeOfTB
0,TRN_0001.nii.gz,1
1,TRN_0002.nii.gz,1
2,TRN_0003.nii.gz,1
3,TRN_0004.nii.gz,1
4,TRN_0005.nii.gz,1
5,TRN_0006.nii.gz,1
6,TRN_0007.nii.gz,4
7,TRN_0008.nii.gz,1
8,TRN_0009.nii.gz,1
9,TRN_0010.nii.gz,1


In [5]:
filenames = df['FileName'].tolist()
num_samples = len(filenames)

labels = df['TypeOfTB'].to_numpy() - 1
stratify = df['TypeOfTB'].to_numpy() - 1
num_classes = labels.max() + 1

labels = tf.one_hot(labels, depth=num_classes)

idxs = [i for i in range(num_samples)]

train_idxs, val_idxs = train_test_split(idxs, test_size=0.2, random_state=random_state, stratify=df['TypeOfTB'].to_numpy() - 1)

del num_classes, idxs, stratify

In [8]:
img_depth = 84

def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan

def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume

def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    
#     print(f"Shape: {img.shape}")
    desired_depth = img_depth
    desired_width = 512
    desired_height = 512
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    
#     img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img


def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(path)
    # Normalize
    volume = normalize(volume)
    # Resize width, height and depth
    volume = resize_volume(volume)
    print(path)
    return volume

In [9]:
def train_f():
    file_names, file_idxs = filenames, train_idxs
    for i, idx in enumerate(file_idxs):
        img_path = os.path.join(dataset_dir, file_names[idx])
        processed = process_scan(img_path)
        
        image = tf.convert_to_tensor(processed, dtype=tf.float16)
        image = image[..., np.newaxis]
        y = labels[idx]
        
        yield image, y
    
def val_f():
    file_names, file_idxs = filenames, val_idxs
    for i, idx in enumerate(file_idxs):
        img_path = os.path.join(dataset_dir, file_names[idx])
        processed = process_scan(img_path)
        
        image = tf.convert_to_tensor(processed, dtype=tf.float16)
        image = image[..., np.newaxis]
        y = labels[idx]
        
        yield image, y

In [10]:
train_batch_size = 1

train_dataset = tf.data.Dataset.from_generator(
                    train_f,
                    (tf.float32, tf.float32),
                    (tf.TensorShape([512, 512, img_depth, 1]), tf.TensorShape([5])))

train_dataset = train_dataset.repeat(None)
train_dataset = train_dataset.batch(train_batch_size)
train_dataset = train_dataset.prefetch(2)


val_batch_size = 1

val_dataset = tf.data.Dataset.from_generator(
                    val_f,
                    (tf.float32, tf.float32),
                    (tf.TensorShape([512, 512, img_depth, 1]), tf.TensorShape([5])))
val_dataset = val_dataset.repeat(None)
val_dataset = val_dataset.batch(val_batch_size)
val_dataset = val_dataset.prefetch(2)

train_steps = int(len(train_idxs) / (train_batch_size * 2))
val_steps = int(len(val_idxs) / (val_batch_size * 3))
# val_steps = 64

In [11]:
# Load Model

model_path = os.path.join(home, 'ImageCLEF2021', 'Models', '3D_CNNs', f'{model_name}.h5')
model = tf.keras.models.load_model(model_path)
# model.trainable = False

# initial_learning_rate = 0.0001
# lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
# )
# model.compile(
#     loss="binary_crossentropy",
#     optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
#     metrics=["acc"],
# )

model.summary()

Model: "3d_image_classification_normalized"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512, 512, 84, 1)] 0         
_________________________________________________________________
3D_64_1 (Conv3D)             (None, 510, 510, 82, 64)  1792      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 255, 255, 41, 64)  0         
_________________________________________________________________
batch_normalization (BatchNo (None, 255, 255, 41, 64)  256       
_________________________________________________________________
3D_64_2 (Conv3D)             (None, 253, 253, 39, 64)  110656    
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 126, 126, 19, 64)  0         
_________________________________________________________________
batch_normalization_1 (Batch (No

In [13]:
train_samples = np.load('train_samples.npz')
train_targets = np.load('train_targets.npz')

val_samples = np.load('val_samples.npz')
val_targets = np.load('val_targets.npz')

In [14]:
train_samples = train_samples['a']
train_targets = train_targets['a']

val_samples = val_samples['a']
val_targets = val_targets['a']

In [15]:
"""
train_preds = model(train_samples)
val_preds = model(val_samples)
"""

train_preds, i = [], 0
for sample in train_samples:
    pred = model(sample[np.newaxis, ...])
    train_preds.append(pred)
    i += 1
    print(f'{i}', end='\r')

print("")    
    
val_preds, i = [], 0
for sample in val_samples:
    pred = model(sample[np.newaxis, ...])
    val_preds.append(pred)
    i += 1
    print(f'{i}', end='\r')

732


In [18]:
print(f'''
\tTrain Samples: {train_samples.shape}
\tTrain Targets: {train_targets.shape}
\tTrain Predictions: {train_preds.shape}
\tT_Preds: {t_preds.shape}
\tT_Targets: {t_targets.shape}
\n
\tVal Samples: {val_samples.shape}
\tVal Targets: {val_targets.shape}
\tVal Predictions: {val_preds.shape}
\tV_Preds: {v_preds.shape}
\tV_Targets: {v_targets.shape}
''')


	Train Samples: (732, 512, 512, 64, 1)
	Train Targets: (732, 5)
	Train Predictions: (732, 5)
	T_Preds: (732,)
	T_Targets: (732,)


	Val Samples: (184, 512, 512, 64, 1)
	Val Targets: (184, 5)
	Val Predictions: (184, 5)
	V_Preds: (184,)
	V_Targets: (184,)



In [19]:
train_kappa = cohen_kappa_score(t_targets, t_preds)
val_kappa = cohen_kappa_score(v_targets, v_preds)

train_acc = accuracy_score(t_targets, t_preds)
val_acc = accuracy_score(v_targets, v_preds)

In [22]:
print(f'''
Training Kappa Score: {train_kappa}\tTraining Accuracy: {train_acc}
Validation Kappa Score: {val_kappa}\tValidation Accuracy: {val_acc}
''')


Training Kappa Score: 0.0	Training Accuracy: 0.11065573770491803
Validation Kappa Score: 0.0	Validation Accuracy: 0.25



In [19]:
model.evaluate(val_dataset, steps=len(val_idxs))

184/184 [==============================] - 932s 5s/step - loss: 0.6995 - acc: 0.8000


[0.6995394825935364, 0.8000015]

In [20]:
valid_preds = np.argmax(validation_preds, axis=1)

In [12]:
validation_preds = model.predict(val_dataset, steps=len(val_idxs))

/home/ayushman.singh/Datasets/ImageCLEF/Dataset/TRN_0162.nii.gz
/home/ayushman.singh/Datasets/ImageCLEF/Dataset/TRN_0852.nii.gz


ResourceExhaustedError:  OOM when allocating tensor with shape[1,510,510,82,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node 3d_image_classification_normalized/3D_64_1/Conv3D (defined at /home/ayushman.singh/anaconda3/envs/tflow/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_distributed_function_3762]

Function call stack:
distributed_function


/home/ayushman.singh/Datasets/ImageCLEF/Dataset/TRN_0146.nii.gz


In [21]:
val_kappa = cohen_kappa_score(v_targets, valid_preds)
val_acc = accuracy_score(v_targets, valid_preds)

In [23]:
print(valid_preds)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [24]:
print(v_targets)

[0 3 1 3 1 4 0 0 0 0 1 1 0 0 0 1 2 1 0 1 1 4 2 0 0 0 0 3 0 3 0 0 4 0 0 2 0
 1 0 1 0 1 0 0 1 0 0 3 1 1 0 2 0 0 1 1 4 4 3 1 1 1 0 0 0 2 0 0 4 0 3 1 1 4
 3 4 0 3 0 0 1 1 4 2 3 0 1 1 0 0 1 4 0 0 2 3 3 1 2 1 1 1 0 1 1 0 0 2 0 1 1
 3 0 3 2 0 0 0 0 0 0 1 0 0 0 0 4 0 0 1 0 1 0 2 0 0 0 4 0 0 0 0 0 0 3 0 0 1
 2 0 2 1 0 2 0 2 0 2 1 3 0 0 4 1 0 0 4 0 1 3 1 0 1 2 0 0 1 1 2 2 3 3 3 2]


In [25]:
print(validation_preds)

[[0.09005298 0.43333203 0.16235366 0.10864624 0.2056151 ]
 [0.09005298 0.43333203 0.16235366 0.10864624 0.2056151 ]
 [0.09005298 0.43333203 0.16235366 0.10864624 0.2056151 ]
 [0.09005298 0.43333203 0.16235366 0.10864624 0.2056151 ]
 [0.09005298 0.43333203 0.16235366 0.10864624 0.2056151 ]
 [0.09005298 0.43333203 0.16235366 0.10864624 0.2056151 ]
 [0.09005298 0.43333203 0.16235366 0.10864624 0.2056151 ]
 [0.09005298 0.43333203 0.16235366 0.10864624 0.2056151 ]
 [0.09005298 0.43333203 0.16235366 0.10864624 0.2056151 ]
 [0.09005298 0.43333203 0.16235366 0.10864624 0.2056151 ]
 [0.09005298 0.43333203 0.16235366 0.10864624 0.2056151 ]
 [0.09005298 0.43333203 0.16235366 0.10864624 0.2056151 ]
 [0.09005298 0.43333203 0.16235366 0.10864624 0.2056151 ]
 [0.09005298 0.43333203 0.16235366 0.10864624 0.2056151 ]
 [0.09005298 0.43333203 0.16235366 0.10864624 0.2056151 ]
 [0.09005298 0.43333203 0.16235366 0.10864624 0.2056151 ]
 [0.09005298 0.43333203 0.16235366 0.10864624 0.2056151 ]
 [0.09005298 0

In [27]:
model.predict((val_samples[0, :, :, :, :])[np.newaxis, ...])

array([[0., 0., 1., 0., 0.]], dtype=float32)

In [28]:
model.predict((val_samples[1, :, :, :, :])[np.newaxis, ...])

array([[0., 0., 1., 0., 0.]], dtype=float32)

In [29]:
model.predict((val_samples[2, :, :, :, :])[np.newaxis, ...])

array([[0., 0., 1., 0., 0.]], dtype=float32)

In [30]:
model.predict((val_samples[3, :, :, :, :])[np.newaxis, ...])

array([[0., 0., 1., 0., 0.]], dtype=float32)

In [31]:
model.predict((val_samples[4, :, :, :, :])[np.newaxis, ...])

array([[0., 0., 1., 0., 0.]], dtype=float32)

In [43]:
# datum = val_dataset.as_numpy_iterator()

# for k in range(0, 6):
#     sample, target = next(datum)
#     print(sample.shape)
#     print(target.shape)
#     print(np.mean(sample))
#     print(model.predict(sample))
#     print(target)

k = 0

for sample, target in val_dataset.take(5):
    print(target)

/home/ayushman.singh/Datasets/ImageCLEF/Dataset/TRN_0162.nii.gz
tf.Tensor([[1. 0. 0. 0. 0.]], shape=(1, 5), dtype=float32)
/home/ayushman.singh/Datasets/ImageCLEF/Dataset/TRN_0162.nii.gz
tf.Tensor([[1. 0. 0. 0. 0.]], shape=(1, 5), dtype=float32)
/home/ayushman.singh/Datasets/ImageCLEF/Dataset/TRN_0162.nii.gz
tf.Tensor([[1. 0. 0. 0. 0.]], shape=(1, 5), dtype=float32)
/home/ayushman.singh/Datasets/ImageCLEF/Dataset/TRN_0162.nii.gz
tf.Tensor([[1. 0. 0. 0. 0.]], shape=(1, 5), dtype=float32)
/home/ayushman.singh/Datasets/ImageCLEF/Dataset/TRN_0162.nii.gz
tf.Tensor([[1. 0. 0. 0. 0.]], shape=(1, 5), dtype=float32)
/home/ayushman.singh/Datasets/ImageCLEF/Dataset/TRN_0162.nii.gz
